In [3]:
#pathToCombined = "D:/Tmp/Coding/datasets/RICO/unique_uis/test_dataset"
traces_prefix = "D:/Tmp/Coding/datasets/RICO/traces/test_traces"
path_to_gesture_json = f"{traces_prefix}/*/trace_*/gestures.json"

In [16]:
import glob
#from pathlib import Path
import json
import preprocessing as pp
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

In [10]:
gesture_json_paths = glob.glob(path_to_gesture_json)
gesture_json_paths

[]

In [11]:
def get_features_from_gestures(gesture_path):
    with tf.io.gfile.GFile(gesture_path) as f:
        gesture_json = json.load(f)
    trace_path = gesture_path.replace('gestures.json', '')
    # Throw away all gestures except the first
    gesture_dict = {gesture[0]:gesture[1][0] for gesture in gesture_json.items()}
    gesture_df = pd.DataFrame(gesture_dict).transpose()
    gesture_df.columns = ['x', 'y']
    return gesture_df, trace_path

In [12]:
keys_to_dtype = {
    #'developer_token_id': tf.int64,
    #'resource_token_id': tf.int64,
    #'screen_caption_token_ids': tf.int64,
    #'appdesc_token_id': tf.int64,
    'clickable_seq': tf.int64,
    'type_id_seq': tf.int64,
    'cord_x_seq': tf.float32,
    'cord_y_seq': tf.float32,
    'visibility_to_user_seq': tf.int64,
    'visibility_seq': tf.int64,
    #'attended_objects': tf.int64,
    #'label_flag': tf.int64,  # 0: padding 1: node
    # 'obj_img_mat': tf.int64,
    'obj_dom_pos': tf.int64,
    #'attention_boxes': tf.float32,
    #'gold_caption': tf.string,
}

In [13]:
def get_features_tensor_from_tree(frame_index, trace_path):
    file_prefix = f"{trace_path}view_hierarchies/{frame_index}"
    features_dict = pp.create_simple_features(file_prefix)
    del features_dict['node_id']
    feature_items = list(features_dict.items())
    features_adapt = to_var_len_feature_dict(feature_items)
    tree_feature_df = pd.DataFrame(features_adapt.items(), columns=['feat', frame_index])
    tree_feature_df = tree_feature_df.set_index('feat')
    tree_feature_df = tree_feature_df.transpose()
    return tree_feature_df
    #return pd.DataFrame(features_adapt, index=[frame_index])

In [14]:
def to_var_len_feature_dict(entries):
    dict = {}
    for feature_item in entries:
        dict[feature_item[0]] = tf.convert_to_tensor(feature_item[1], dtype=keys_to_dtype[feature_item[0]])
    return dict

In [15]:
gesture_path = gesture_json_paths[1]
gesture_df, trace_path = get_features_from_gestures(gesture_path)
frameIndex = gesture_df.index.values[1]
feature_df = get_features_tensor_from_tree(frameIndex, trace_path)
feature_df
#feature_item = features[0]
#feature_item
#tf.convert_to_tensor(feature_item[1], dtype=keys_to_dtype[feature_item[0]])

IndexError: list index out of range

In [ ]:
# gesture_df['x'].dtype # float64
# gesture_df['y'].dtype # float64

In [ ]:
for t,v in keys_to_dtype.items():
    print(feature_df[t]['4758'][0].dtype)

In [ ]:
all_feat_df = pd.concat([gesture_df, feature_df], axis=1)
all_feat_df

In [ ]:
feature_df['type_id_seq'].iloc[0][0].dtype

In [ ]:
 # Get set of frames for each screen, but don't mix them during training LSTM
all_features_df = pd.DataFrame()
for trace_id, gesture_path in enumerate(gesture_json_paths):
    gesture_df, trace_path = get_features_from_gestures(gesture_path)
    tree_features_df = pd.DataFrame()
    for frame_id in gesture_df.index.values:
        tree_feature_df = get_features_tensor_from_tree(frame_id, trace_path)
        tree_features_df = pd.concat([tree_features_df, tree_feature_df], axis=0)
    trace_features_df = pd.concat([gesture_df, tree_features_df], axis=1)
    trace_features_df['trace_id'] = trace_id
    all_features_df = pd.concat([all_features_df, trace_features_df], axis=0)
    
all_features_df.head()
        

In [ ]:
# TODO: split by trace id
df = all_features_df
test_size = 6
test_ind = len(df) - test_size

train = df.iloc[:test_ind]
test = df.iloc[test_ind:]

In [ ]:
#df['obj_dom_pos'].dtype

In [ ]:
from keras.layers import Normalization

scaler = Normalization()
scaler_invert = Normalization(invert=True)

scaler.adapt(train)
scaler_invert.adapt(train)

scaled_train = scaler(train)
scaled_test = scaler(test)

print(scaler_invert(scaled_train))

In [ ]:
for baseName in baseNames:
    file_prefix = f"{pathToCombined}/{baseName}.json"
    features = pp.create_simple_features(file_prefix)